### setup

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [13]:
import datetime as dt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

### read data

In [7]:
df = pd.read_csv("data/raw/hurricanes/events-US-1980-2022.csv", skiprows=1)

In [9]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [12]:
df["date"] = pd.to_datetime(df["begin_date"], format="%Y%m%d")

In [14]:
df["year"] = df["date"].dt.year

In [23]:
df["cost"] = df["total_cpi-adjusted_cost_(millions_of_dollars)"].str.replace(
    "TBD", "NaN"
)

In [24]:
df["cost"] = df["cost"].astype(float)

In [26]:
hurricane_costs = (
    df[df["disaster"] == "Tropical Cyclone"][["year", "cost"]]
    .groupby("year")
    .agg(sum)
    .reset_index()
)

In [28]:
# dw.create_chart(
#    "cost of hurricanes over time", chart_type="d3-area", data=hurricane_costs
# )
chart_id = "Xc2gB"

In [29]:
#checking numbers are correct
hurricanes = df[df["disaster"] == "Tropical Cyclone"]

In [34]:
hurricanes[hurricanes["year"] == 2017]

In [36]:
hurricanes.sort_values("cost", ascending=False)

,name,disaster,begin_date,end_date,total_cpi-adjusted_cost_(millions_of_dollars),deaths,date,year,cost
115,Hurricane Katrina (August 2005),Tropical Cyclone,20050825,20050830,186294.0,1833,2005-08-25,2005,186294.0
247,Hurricane Harvey (August 2017),Tropical Cyclone,20170825,20170831,148750.0,89,2017-08-25,2017,148750.0
249,Hurricane Maria (September 2017),Tropical Cyclone,20170919,20170921,107100.0,2981,2017-09-19,2017,107100.0
186,Hurricane Sandy (October 2012),Tropical Cyclone,20121030,20121031,81950.0,159,2012-10-30,2012,81950.0
317,Hurricane Ida (August 2021),Tropical Cyclone,20210829,20210901,78722.3,96,2021-08-29,2021,78722.3
248,Hurricane Irma (September 2017),Tropical Cyclone,20170906,20170912,59500.0,97,2017-09-06,2017,59500.0
42,Hurricane Andrew (August 1992),Tropical Cyclone,19920823,19920827,55890.0,61,1992-08-23,1992,55890.0
141,Hurricane Ike (September 2008),Tropical Cyclone,20080912,20080914,40198.5,112,2008-09-12,2008,40198.5
111,Hurricane Ivan (September 2004),Tropical Cyclone,20040912,20040921,31570.9,57,2004-09-12,2004,31570.9
264,Hurricane Michael (October 2018),Tropical Cyclone,20181010,20181011,28971.0,49,2018-10-10,2018,28971.0


In [32]:
hurricanes.groupby("year").agg(annual_cost=pd.NamedAgg(column="cost", aggfunc=sum))

,annual_cost
year,
1980,2071.0
1983,8760.0
1985,9853.6
1989,22440.4
1991,3216.3
1992,62307.0
1994,1967.0
1995,14543.6
1996,9263.0
